In [12]:
a = 1 if False else 2

a

2

In [4]:
solution = lambda a, b: (b*2).find(a)
solution('hello', 'ohell')

1

In [11]:
a, b, c = 1, 2
b

ValueError: not enough values to unpack (expected 3, got 2)

In [14]:
a = 0.9
round(a)

1